<a href="https://colab.research.google.com/github/codebuzer/Bioactivity-Prediction-App/blob/main/Ml_Assign_3_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Import required libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
import wandb
from google.colab import drive
import os

# Performance optimizations for faster training
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# Mount Google Drive
drive.mount('/content/drive')
# Configure matplotlib
%matplotlib inline
plt.close('all')
import datetime
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
class LightCNN(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(LightCNN, self).__init__()

        self.features = nn.Sequential(
            # First block
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Second block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Third block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 100)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

def get_cifar100_classes():
    return [
        'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle',
        'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel',
        'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock',
        'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur',
        'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster',
        'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
        'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
        'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
        'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
        'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea',
        'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
        'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
        'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
        'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
        'worm'
    ]


In [9]:
def load_and_prepare_data(batch_size):
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomAffine( degrees=0,  translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                            download=True, transform=train_transform)
    testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                           download=True, transform=test_transform)

    trainset.classes = get_cifar100_classes()
    testset.classes = get_cifar100_classes()

    train_size = int(0.9 * len(trainset))
    val_size = len(trainset) - train_size
    trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

    train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True,
                            num_workers=2, pin_memory=True)
    val_loader = DataLoader(valset, batch_size=batch_size, shuffle=False,
                          num_workers=2, pin_memory=True)
    test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False,
                           num_workers=2, pin_memory=True)

    return train_loader, val_loader, test_loader

def show_random_images(dataset, run, num_classes=10, images_per_class=5):
    """Modified to use existing wandb run"""
    if hasattr(dataset, 'dataset'):
        classes = dataset.dataset.classes if hasattr(dataset.dataset, 'classes') else None
    else:
        classes = dataset.classes if hasattr(dataset, 'classes') else None

    fig = plt.figure(figsize=(15, 30))

    selected_classes = np.random.choice(100, num_classes, replace=False)
    class_images = {i: [] for i in selected_classes}

    for img, label in dataset:
        if label in selected_classes and len(class_images[label]) < images_per_class:
            class_images[label].append(img)
        if all(len(imgs) == images_per_class for imgs in class_images.values()):
            break

    for idx, class_idx in enumerate(selected_classes):
        for j, img in enumerate(class_images[class_idx]):
            plt.subplot(num_classes, images_per_class, idx * images_per_class + j + 1)
            img = img.numpy().transpose(1, 2, 0)
            img = img * np.array([0.5, 0.5, 0.5]) + np.array([0.5, 0.5, 0.5])
            img = np.clip(img, 0, 1)
            plt.imshow(img)
            plt.axis('off')
            if j == 0:
                if classes is not None:
                    plt.title(f'Class: {classes[class_idx]}')
                else:
                    plt.title(f'Class {class_idx}')

    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/ML Assignment 3/random_images.png')
    run.log({"dataset_samples": wandb.Image(plt)})
    plt.close()


In [10]:
def validate_model(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss = running_loss / len(val_loader)
    val_acc = 100. * correct / total

    return val_loss, val_acc

def train_and_log(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, run):
    """Modified training function to use existing wandb run"""
    scaler = torch.cuda.amp.GradScaler()
    best_val_acc = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            optimizer.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            if i % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}] Batch [{i}/{len(train_loader)}]')

        train_loss = running_loss / len(train_loader)
        train_acc = 100. * correct / total

        val_loss, val_acc = validate_model(model, val_loader, criterion, device)

        # Log metrics
        run.log({
            "epoch": epoch,
            "train/loss": train_loss,
            "train/accuracy": train_acc,
            "val/loss": val_loss,
            "val/accuracy": val_acc
        })

        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(),
                      f'/content/drive/MyDrive/ML Assignment 3/best_model_{run.id}.pth')

    return model


def generate_classification_report(model, test_loader, device, config, save_path, run):
    """Modified to use existing wandb run"""
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    class_names = get_cifar100_classes()
    report = classification_report(all_labels, all_preds,
                                 target_names=class_names,
                                 digits=4)

    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    report_filename = f'classification_report_{config["optimizer"]}_lr_{config["lr"]}_{timestamp}.txt'
    report_path = os.path.join(save_path, report_filename)

    with open(report_path, 'w') as f:
        f.write(f"Configuration:\n{config}\n\n")
        f.write(report)

    # Log to wandb
    run.log({
        "classification_report": wandb.Table(
            columns=["Class", "Precision", "Recall", "F1-Score", "Support"],
            data=[[class_names[i], *map(float, row.strip().split()[1:-1]),
                  int(row.strip().split()[-1])]
                 for i, row in enumerate(report.split('\n')[2:-5])]
        )
    })

    # Create and log confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(20, 20))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    cm_filename = f'confusion_matrix_{config["optimizer"]}_lr_{config["lr"]}_{timestamp}.png'
    cm_path = os.path.join(save_path, cm_filename)
    plt.savefig(cm_path)
    run.log({"confusion_matrix": wandb.Image(cm_path)})
    plt.close()

    return report

def main():
    # Login to wandb
    wandb.login()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    reports_dir = '/content/drive/MyDrive/ML Assignment 3/cifar100_reports'
    os.makedirs(reports_dir, exist_ok=True)

    configs = [
        {"lr": 0.001, "batch_size": 128, "optimizer": "adam", "dropout": 0.3, "epochs": 20},
        {"lr": 0.01, "batch_size": 32, "optimizer": "sgd", "dropout": 0.3, "epochs": 20},
        {"lr": 0.0001, "batch_size": 256, "optimizer": "adam", "dropout": 0.7,"epochs":20},
        {"lr": 0.005, "batch_size": 128, "optimizer": "sgd", "dropout": 0.4,"epochs":20},
        {"lr": 0.002, "batch_size": 32, "optimizer": "adam", "dropout": 0.6,"epochs":20}
    ]

    for config in configs:
        # Initialize wandb run at the start of each configuration
        with wandb.init(project="cifar100-cnn", config=config, reinit=True) as run:
            print(f"\nTraining with config: {config}")

            train_loader, val_loader, test_loader = load_and_prepare_data(config["batch_size"])

            # Show random images at the start
            print("Generating random image samples...")
            show_random_images(train_loader.dataset, run=run)

            model = LightCNN(dropout_rate=config["dropout"]).to(device)
            criterion = nn.CrossEntropyLoss()

            if config["optimizer"] == "adam":
                optimizer = optim.Adam(model.parameters(), lr=config["lr"])
            else:
                optimizer = optim.SGD(model.parameters(), lr=config["lr"],
                                    momentum=0.9, nesterov=True)

            # Train model
            model = train_and_log(model, train_loader, val_loader, criterion,
                                optimizer, config["epochs"], device, run)

            # Generate classification report
            print("\nGenerating classification report...")
            report = generate_classification_report(model, test_loader, device,
                                                 config, reports_dir, run)
            print("\nClassification Report:")
            print(report)
            print(f"\nReport saved to {reports_dir}")

if __name__ == "__main__":
    main()



Using device: cuda



Training with config: {'lr': 0.001, 'batch_size': 128, 'optimizer': 'adam', 'dropout': 0.3, 'epochs': 20}
Generating random image samples...


<ipython-input-10-0b4a4a8ed1dd>:25: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
<ipython-input-10-0b4a4a8ed1dd>:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch [1/20] Batch [0/352]
Epoch [1/20] Batch [100/352]
Epoch [1/20] Batch [200/352]
Epoch [1/20] Batch [300/352]
Epoch [1/20]
Train Loss: 3.9664, Train Acc: 8.92%
Val Loss: 3.6096, Val Acc: 14.04%
Epoch [2/20] Batch [0/352]
Epoch [2/20] Batch [100/352]
Epoch [2/20] Batch [200/352]
Epoch [2/20] Batch [300/352]
Epoch [2/20]
Train Loss: 3.4936, Train Acc: 16.12%
Val Loss: 3.3227, Val Acc: 19.40%
Epoch [3/20] Batch [0/352]
Epoch [3/20] Batch [100/352]
Epoch [3/20] Batch [200/352]
Epoch [3/20] Batch [300/352]
Epoch [3/20]
Train Loss: 3.2748, Train Acc: 19.91%
Val Loss: 3.0896, Val Acc: 23.94%
Epoch [4/20] Batch [0/352]
Epoch [4/20] Batch [100/352]
Epoch [4/20] Batch [200/352]
Epoch [4/20] Batch [300/352]
Epoch [4/20]
Train Loss: 3.1166, Train Acc: 22.74%
Val Loss: 2.9759, Val Acc: 25.70%
Epoch [5/20] Batch [0/352]
Epoch [5/20] Batch [100/352]
Epoch [5/20] Batch [200/352]
Epoch [5/20] Batch [300/352]
Epoch [5/20]
Train Loss: 3.0206, Train Acc: 24.47%
Val Loss: 2.8176, Val Acc: 29.50%
Epoch 

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train/accuracy,▁▃▄▄▅▅▆▆▆▇▇▇▇▇▇▇████
train/loss,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
val/accuracy,▁▂▄▄▅▅▆▆▆▆▇▇▇▇▇▇████
val/loss,█▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁
epoch,19
train/accuracy,37.04222
train/loss,2.39777
val/accuracy,40.1
val/loss,2.27199


...

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train/accuracy,▁▂▃▄▅▅▅▆▆▆▇▇▇▇▇▇████
train/loss,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
val/accuracy,▁▂▃▄▄▅▅▆▅▅▆▆▆▇▇▇▇▇██
val/loss,█▇▆▅▅▄▄▃▃▃▃▂▃▂▂▂▂▂▁▁
epoch,19
train/accuracy,35.11111
train/loss,2.50362
val/accuracy,39.94
val/loss,2.33548



Training with config: {'lr': 0.002, 'batch_size': 32, 'optimizer': 'adam', 'dropout': 0.6, 'epochs': 20}
Generating random image samples...


<ipython-input-10-0b4a4a8ed1dd>:25: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
<ipython-input-10-0b4a4a8ed1dd>:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch [1/20] Batch [0/1407]
Epoch [1/20] Batch [100/1407]
Epoch [1/20] Batch [200/1407]
Epoch [1/20] Batch [300/1407]
Epoch [1/20] Batch [400/1407]
Epoch [1/20] Batch [500/1407]
Epoch [1/20] Batch [600/1407]
Epoch [1/20] Batch [700/1407]
Epoch [1/20] Batch [800/1407]
Epoch [1/20] Batch [900/1407]
Epoch [1/20] Batch [1000/1407]
Epoch [1/20] Batch [1100/1407]
Epoch [1/20] Batch [1200/1407]
Epoch [1/20] Batch [1300/1407]
Epoch [1/20] Batch [1400/1407]
Epoch [1/20]
Train Loss: 4.4675, Train Acc: 2.58%
Val Loss: 4.2213, Val Acc: 5.76%
Epoch [2/20] Batch [0/1407]
Epoch [2/20] Batch [100/1407]
Epoch [2/20] Batch [200/1407]
Epoch [2/20] Batch [300/1407]
Epoch [2/20] Batch [400/1407]
Epoch [2/20] Batch [500/1407]
Epoch [2/20] Batch [600/1407]
Epoch [2/20] Batch [700/1407]
Epoch [2/20] Batch [800/1407]
Epoch [2/20] Batch [900/1407]
Epoch [2/20] Batch [1000/1407]
Epoch [2/20] Batch [1100/1407]
Epoch [2/20] Batch [1200/1407]
Epoch [2/20] Batch [1300/1407]
Epoch [2/20] Batch [1400/1407]
Epoch [2/20

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train/accuracy,▁▂▂▃▄▄▅▅▅▆▆▆▆▇▇▇████
train/loss,█▇▆▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁
val/accuracy,▁▂▂▃▄▄▅▅▆▆▆▆▆▇▇▇████
val/loss,█▇▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁
epoch,19
train/accuracy,18.48
train/loss,3.36807
val/accuracy,26.06
val/loss,3.00249


In [ ]:
#https://wandb.ai/abuzarkhan-indraprastha-institute-of-information-technol/cifar100-cnn/runs/4q9gkawj?nw=nwuserabuzarkhan